In [ ]:
import pickle
import numpy as np
import msmrd2
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.markovModels import msmrdMarkovStateModel as msmrdMSM
from msmrd2.integrators import msmrdIntegrator 

In [ ]:
# Main parameters
numBoundStates = 2
maxNumBoundStates = 10
relativeDistanceCutOff = 2.2
numParticleTypes = 1 # num. of particle types (not states) in unbound state
boxsize = 6

# Discretization parameters (need to be consistent with those of integrator
numSphericalSectionsPos = 7
numRadialSectionsQuat = 5
numSphericalSectionsQuat = 7
totalnumSecsQuat = numSphericalSectionsQuat*(numRadialSectionsQuat -1) + 1
numTransitionsStates = numSphericalSectionsPos * totalnumSecsQuat #203 

# Trajectory relevant parameters (Need to be equal to the one used to generate the rateDictionary)
dt = 0.00001
lagtime = 200 
effectiveLagtime = dt*lagtime
# Other trajectory parameters for reference.
timesteps = 4000000 # 2000000
stride = 25 #2500
maxentries = int(timesteps/stride)

In [ ]:
# Load pickled rateDicitionary generated in generateRateDictionary
pickle_in = open("data/ratedictionary_dimer.pickle","rb")
rateDictionary = pickle.load(pickle_in)

In [ ]:
# Define continuous-time MSM for unbound dynamics (same for all particles)
MSMtype = 0
ratematrix = np.array([[0]]) # no unbound dynamics
seed = 0 # Seed = -1 used random device as seed
unboundMSM = ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([1.0])
Drotlist = np.array([1.0])
unboundMSM.setD(Dlist)
unboundMSM.setDrot(Drotlist)

In [ ]:
# Define coupling Markov model
seed = 0
Dbound = [0.5, 0.5]
DboundRot = [1.0, 1.0]
couplingMSM = msmrdMSM(effectiveLagtime, numBoundStates, numTransitionsStates, seed, rateDictionary) 
couplingMSM.setDbound(Dbound, DboundRot)
couplingMSM.setmaxNumberBoundStates(maxNumBoundStates)

In [ ]:
# Define particle list (make sure all particles are inside box)
ptype = 0
pstate = 0
D = Dlist[pstate]
Drot = Drotlist[pstate]
position = np.array([1,1,1])
orientation = np.array([1,0,0,0])
part1 = msmrd2.particleMS(ptype, pstate, D, Drot, 0.1*position, orientation)
part2 = msmrd2.particleMS(ptype, pstate, D, Drot, -0.1*position, orientation)
part1.deactivateMSM()
part2.deactivateMSM()
partList = msmrd2.integrators.particleMSList([part1, part2])

In [ ]:
# MSM/RD integrator definition
dt = 0.002 # should be smaller than Gillespie inverse transition rates
seed = 0 # Seed = -1 used random device as seed
bodytype = 'rigidbody'
integrator = msmrdIntegrator(dt, seed, bodytype, numParticleTypes, relativeDistanceCutOff, unboundMSM, couplingMSM)

In [ ]:
# Set boundary and discretization in integrator (optional since default values also work)
boundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')
#boundary = msmrd2.sphere(boxsize,'reflective')
#boundary = msmrd2.noBoundary()
integrator.setBoundary(boundary)
discretization = msmrd2.discretizations.positionOrientationPartition(relativeDistanceCutOff, numSphericalSectionsPos,
                                                                    numRadialSectionsQuat, numSphericalSectionsQuat)
integrator.setDiscretization(discretization)        include/trajectories/discrete/patchyDimer.hpp
        include/trajectories/discrete/patchyProtein.hpp

In [ ]:
print(partList[0].position, partList[1].position, 
      partList[0].state, partList[1].state, 
      partList[0].boundTo, partList[1].boundTo)
prevstate = 1

In [ ]:
for i in range(20000):
    integrator.integrate(partList)
    if (partList[0].state != prevstate):
        print("{:.4f}".format(integrator.clock), partList[0].state, partList[1].state)
        #print(partList[0].position, partList[1].position, partList[0].state, partList[1].state, partList[0].boundTo, partList[1].boundTo)
        prevstate = partList[0].state

In [ ]:
print(partList[0].position, partList[1].position, 
      partList[0].state, partList[1].state, 
      partList[0].boundTo, partList[1].boundTo)

In [ ]:
rateDictionary

In [ ]:
couplingMSM.computeTransition2BoundState(180)

In [ ]:
couplingMSM.computeTransition2UnboundState(2)